In [1]:
# add source to apt
# sudo apt update
# sudo apt install ni-daqmx


import nidaqmx


In [ ]:
import numpy as np
ramp = np.linspace(0,0.25,1000)

In [ ]:
with nidaqmx.Task() as task:
    print(task.timing)
    channel = task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
    print(task.timing.samp_clk_rate)
    #channel.ao_func_gen_freq = 100000
    task.write(ramp, auto_start=True)

1000.0


In [7]:
import nidaqmx.stream_writers as stream_writers


with nidaqmx.Task() as task:
    print(task.timing)
    #channel = task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    channel = task.ao_channels.add_ao_voltage_chan("Dev1/ao0")

    task.timing.cfg_samp_clk_timing(rate=100,
                                        #source='/cDAQ1/ao/SampleClock',
                                        #active_edge=nidaqmx.constants.Edge.RISING,
                                        #sample_mode=nidaqmx.constants.AcquisitionType.FINITE,
                                        #samps_per_chan=1000
                                   )
    writer = stream_writers.AnalogMultiChannelWriter(task.out_stream)

    task.write(ramp, auto_start=False)
    task.start()
    task.wait_until_done()


In [26]:
def callback(task_handle, every_n_samples_event_type,
        number_of_samples, callback_data):
    print(number_of_samples)
    return 0
from nidaqmx.stream_readers import AnalogSingleChannelReader

with nidaqmx.Task() as task:
    print(task.timing)
    #channel = task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    task.ai_channels.add_ai_voltage_chan('Dev1/ai0')
    task.timing.cfg_samp_clk_timing(rate=1000,
                                        #source='/cDAQ1/ao/SampleClock',
                                        active_edge=nidaqmx.constants.Edge.RISING,
                                        sample_mode=nidaqmx.constants.AcquisitionType.FINITE,
                                        samps_per_chan=1000)
    reader = nidaqmx.stream_readers.AnalogMultiChannelReader(task.in_stream)
    task.in_stream.configure_logging('logfile')
    task.start()
    task.wait_until_done()
        

In [13]:
from ctypes.util import find_library
print(find_library('nidaqmx'))

libnidaqmx.so.1


In [12]:
with nidaqmx.Task() as task:
    ao = task.ao_channels
    print(ao.channel_names)

[]


In [20]:
with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    a = task.read(number_of_samples_per_channel=10)
print(a)

[0.24987994869128605, 0.24987994869128605, 0.24955805741794146, 0.24955805741794146, 0.24987994869128605, 0.24923616614469118, 0.24955805741794146, 0.24955805741794146, 0.25020183996472506, 0.24987994869128605]


In [37]:
import nidaqmx
from nidaqmx import constants
from nidaqmx import stream_readers
from nidaqmx import stream_writers
import matplotlib.pyplot as plt

import numpy as np

#user input Acquisition
Ch00_name = 'A00'
Sens_Ch00 = 100#sensibilidade em mV/g
Ch01_name = 'A01'
Sens_Ch01 = 100#sensibilidade em mV/g
num_channels = 2
fs_acq = 300 #sample frequency
read_freq = 30
t_med = 1000 #multiples in buffer

n = []

with nidaqmx.Task() as task:
    
    #task.ai_channels.add_ai_accel_chan(physical_channel="Dev1/ai0", name_to_assign_to_channel=Ch00_name,
    #                                   sensitivity=Sens_Ch00, min_val=-5, max_val=5, current_excit_val=0.002)
    #task.ai_channels.add_ai_accel_chan(physical_channel="cDAQ9191-1B7B393Mod1/ai1", name_to_assign_to_channel=Ch01_name,
    #                                   sensitivity=Sens_Ch01, min_val=-5, max_val=5, current_excit_val=0.002)
    task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai1")

    task.timing.cfg_samp_clk_timing(rate=fs_acq, 
                                    sample_mode=constants.AcquisitionType.CONTINUOUS, 
                                    samps_per_chan=(t_med * fs_acq),) # you may not need samps_per_chan

    # I set an input_buf_size
    samples_per_buffer = int(fs_acq // read_freq)  # 30 hz update
    # task.in_stream.input_buf_size = samples_per_buffer * 10  # plus some extra space

    reader = stream_readers.AnalogMultiChannelReader(task.in_stream)
    writer = stream_writers.AnalogMultiChannelWriter(task.out_stream)

    def reading_task_callback(task_idx, event_type, num_samples, callback_data=None):
        """After data has been read into the NI buffer this callback is called to read in the data from the buffer.

        This callback is for working with the task callback register_every_n_samples_acquired_into_buffer_event.

        Args:
            task_idx (int): Task handle index value
            event_type (nidaqmx.constants.EveryNSamplesEventType): ACQUIRED_INTO_BUFFER
            num_samples (int): Number of samples that was read into the buffer.
            callback_data (object)[None]: No idea. Documentation says: The callback_data parameter contains the value
                you passed in the callback_data parameter of this function.
        """
        print(num_samples,event_type,num_channels)
        buffer = np.zeros((num_channels, num_samples), dtype=np.float64)
        #print(num_channels, buffer)
        try:
            reader.read_many_sample(buffer, num_samples, timeout=3)
        except Exception as e:
            print('failed')
            print(e)
        print(buffer)
        global n
        n.append(buffer)
        # Do something with the data
        return 0

    task.register_every_n_samples_acquired_into_buffer_event(samples_per_buffer, reading_task_callback)
    task.start()
    task.wait_until_done(timeout = 5)


10 1 2
[[0.249719   0.24987995 0.24939711 0.24987995 0.25004089 0.249719
  0.25004089 0.249719   0.249719   0.249719  ]
 [0.16956808 0.20626368 0.22686472 0.23813092 0.24472969 0.24859238
  0.25036279 0.25148941 0.25197224 0.25293792]]


sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1a0224acb20>


DaqError: Wait Until Done did not indicate that the task was done within the specified timeout.
Increase the timeout, check the program, and make sure connections for external timing and triggering are in place.
Task Name: _unnamedTask<1C>

Status Code: -200560

In [40]:
print(n)

[array([[0.249719  , 0.24987995, 0.24939711, 0.24987995, 0.25004089,
        0.249719  , 0.25004089, 0.249719  , 0.249719  , 0.249719  ],
       [0.16956808, 0.20626368, 0.22686472, 0.23813092, 0.24472969,
        0.24859238, 0.25036279, 0.25148941, 0.25197224, 0.25293792]])]


In [43]:
import pprint
import nidaqmx
from nidaqmx.constants import AcquisitionType

pp = pprint.PrettyPrinter(indent=4)


with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("Dev1/ai0")

    task.timing.cfg_samp_clk_timing(1000, sample_mode=AcquisitionType.CONTINUOUS)
        samples = []

    def callback(task_handle, every_n_samples_event_type,
                 number_of_samples, callback_data):
        #print('Every N Samples callback invoked.')
        samples.extend(task.read(number_of_samples_per_channel=1000))
        return 0

    task.register_every_n_samples_acquired_into_buffer_event(
        1000, callback)

    task.start()

    input('Running task. Press Enter to stop and see number of '
          'accumulated samples.\n')

    print(len(samples))

Running task. Press Enter to stop and see number of accumulated samples.
 


1000


In [44]:
print(samples)

[0.24971900305460198, 0.24987994869128605, 0.24955805741794146, 0.24955805741794146, 0.2500408943279937, 0.2500408943279937, 0.24971900305460198, 0.24987994869128605, 0.2500408943279937, 0.2500408943279937, 0.24971900305460198, 0.24971900305460198, 0.24971900305460198, 0.24987994869128605, 0.24971900305460198, 0.24987994869128605, 0.24987994869128605, 0.24971900305460198, 0.24955805741794146, 0.2500408943279937, 0.2500408943279937, 0.24987994869128605, 0.2500408943279937, 0.24987994869128605, 0.2500408943279937, 0.24987994869128605, 0.2500408943279937, 0.24987994869128605, 0.24971900305460198, 0.24971900305460198, 0.24971900305460198, 0.24971900305460198, 0.24971900305460198, 0.25020183996472506, 0.24987994869128605, 0.24987994869128605, 0.24987994869128605, 0.24987994869128605, 0.24955805741794146, 0.24987994869128605, 0.24971900305460198, 0.24971900305460198, 0.24971900305460198, 0.24971900305460198, 0.24987994869128605, 0.24987994869128605, 0.2500408943279937, 0.24971900305460198, 0